In [10]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm import tqdm_pandas
pd.set_option( 'display.max_rows' ,  2000 )
pd.set_option( 'display.max_columns' ,  2000 )
pd.set_option( 'display.width' ,  200 )

## 1. load_data

In [2]:
#### 데이터 경로 설정
file_path = 'data/test.csv'
raw_data = pd.read_csv(file_path)


# 2.DATA Preprocessing

#### 게임별 데이터 프레임 만들기
    1.게임별 플레이어 event 컬럼화
    2.플레어 간의 event 비율화
        rate_action = player1 action / ( player0 action + player1 action )
            

####  - 게임별 기본 컬럼 생성 함수
    game_id, playtime , winner , species , camera , ability , selection , right click , setControlGroup 
    AddToControlGroup , ControlGroup , rate_action

In [35]:
def get_BaseColumn (DATA):
    
# 기본 dataframe 틀 만들기
## game_id , winner , p0_species , p0_Camera , p0_Ability , p0_Selection, p0_Right Click, p0_SetControlGroup,p0_GetControlGroup, p0_AddToControlGroup , p0_ControlGroup

    unique_id = DATA['game_id'].unique()
    unique_id.sort()

    df = {'game_id':[] , 'p0_species' : [] , 'p1_species':[] , 'playtime': []} #'winner':[] ,

    events_uni = list(DATA['event'].unique())
    p0_events = [ 'p0_' + i  for i in events_uni ]
    p1_events = [ 'p1_' + i  for i in events_uni ]
    p0_events.extend(p1_events)
    t_events = p0_events
    t_events

    for name in (t_events):
        df.update({name:[]})

        
# 게임별 데이터 insert 하기

    for id_ in tqdm(unique_id):
        one_game = DATA.loc[DATA.game_id == id_ ]

        
        # playtime
        df['playtime'].append(one_game['time'].values[-1])
        # game_id
        df['game_id'].append(id_)
        # 0플레이어 종족
        df['p0_species'].append(one_game.loc[one_game['player'] == 0 , 'species' ].values[0])
        # 1플레이어 종족
        df['p1_species'].append(one_game.loc[one_game['player'] == 1 , 'species' ].values[0])
        # 승리 플레이어
       # df['winner'].append(one_game['winner'].values[0])


        # 플레이어 이벤트 value_counts
        event_counts = one_game.groupby(['player'])['event'].value_counts()
        
        for i in events_uni: # 0플레이어 이벤트 넣기
            try:
                df['p0_'+i].append(event_counts[0][i])

            except:
                df['p0_'+i].append(0)


        for i in events_uni: # 1플레이어 이벤트 넣기
            try:
                df['p1_'+i].append(event_counts[1][i])

            except:
                df['p1_'+i].append(0)
        
        print(id_ , end = '\r')
         

    df = pd.DataFrame(df)
   

    for action in events_uni:
        try:
            df['rate_'+action] = df['p1_'+action ] / (df['p1_'+action ] + df['p0_'+action ])
        except:
            print('안대여')

    df.fillna(0 , inplace = True )
    print('BaseColumn_Complete')
    return df


#### - 일꾼 수 구하기

In [31]:
def make_worker( train , p ) :
    p = int(p)
    scv = train.loc[(train['event']=='Ability') & (train['event_contents']=='(1360) - TrainSCV') & (train['player']==p)]
    probe = train.loc[(train['event']=='Ability') & (train['event_contents']=='(15E0) - TrainProbe') & (train['player']==p)]
    drone = train.loc[(train['event']=='Ability') & (train['event_contents']=='(1820) - MorphDrone') & (train['player']==p)]
    print(f'{p}player SCV go sir')
    

    game_ids = train['game_id'].unique()
    game_ids.sort()
    train_num=[]
    for game_id in game_ids:
        a=scv[(scv.game_id==game_id)]
        train_num.append(len(a))
    train_num
    number1 = pd.DataFrame(train_num,columns=['worker_scv'])

    game_ids = train['game_id'].unique()
    game_ids.sort()
    train_num=[]
    for game_id in game_ids:
        a=probe[probe.game_id==game_id]
        train_num.append(len(a))
    number2 = pd.DataFrame(train_num,columns=['worker_probe'])
    
    
    game_ids = train['game_id'].unique()
    game_ids.sort()
    train_num=[]
    for game_id in game_ids:
        a=drone[drone.game_id==game_id]
        train_num.append(len(a))
    number3 = pd.DataFrame(train_num,columns=['worker_drone'])

    number = pd.concat([number1,number2,number3],axis=1)


    number['workers_p'+str(p)] = number["worker_scv"] + number["worker_probe"] + number["worker_drone"]
    total = number['workers_p'+str(p)]
    
    print(f'{p} worker Complete')
    return total



def get_worker( raw_data ):
    p0_worker=make_worker( raw_data , 0 )
    p1_worker=make_worker( raw_data , 1 )
    
    
    game_id = raw_data['game_id'].unique()
    game_id.sort()

    game_id = pd.Series(game_id)
    res = pd.concat( [ p0_worker , p1_worker , game_id ] , axis = 1)
    res = res.rename( columns = {0: 'game_id'})
    
    return res

#### - 공격횟수 찾기

In [27]:
def get_Attack( raw_data ):
    abilities=raw_data.loc[raw_data['event'] == 'Ability']
    attk=abilities.loc[abilities['event_contents'].str.contains('Attack'),:]

    unique_id = raw_data['game_id'].unique()
    unique_id.sort()


    p0_atk=[]
    for i in unique_id :
        q2w=len(attk.loc[(attk['game_id']==i)&(attk['player']==0), :])
        p0_atk.append(q2w)


    p1_atk=[]
    for i in unique_id :
        q3w=len(attk.loc[(attk['game_id']==i)&(attk['player']==1), :])
        p1_atk.append(q3w)    


    attks=[]
    for i in unique_id:
        a=len(attk.loc[attk['game_id']==i])
        attks.append(a)
    
    df = {'p0_attack' : p0_atk , 'p1_attack' : p1_atk , 'total_attack' : attks }
    
    return pd.DataFrame(df)

#### - 데이터 병합하기

In [38]:
def get_Game_df( raw_data ): 
    base_column = get_BaseColumn(raw_data)
    worker = get_worker(raw_data)
    num_attack = get_Attack( raw_data )
    res=pd.concat( [ base_column , worker , num_attack] , axis = 1 )
    res.set_index('game_id', inplace=True)
    return res

### 구해보, 자.

In [34]:
df = get_Game_df(raw_data)
df.to_csv('preprocessed_data.csv' , index= False , header = True )



100%|██████████| 16/16 [00:00<?, ?it/s]


BaseColumn_Complete
0player SCV go sir
0 worker Complete
1player SCV go sir
1 worker Complete
